In [1]:
import os
from pathlib import Path
import pandas as pd
from newsapi import NewsApiClient
import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from dotenv import load_dotenv
load_dotenv()

True

In [3]:
# news api for collecting data
api_key = os.getenv("news_api")
newsapi = NewsApiClient(api_key=api_key)

In [6]:
# import our initial batch of 20 companies (10 bankrupt, 10 healthy)
filepath = Path("../Project_2/CompanyList_2.csv")
bankrupt_df = pd.read_csv(filepath, parse_dates=True, infer_datetime_format=True)
bankrupt_df.head(20)

,Company,Industry,Bankrupty Filed Date,Binary
0,JC Penny,Retail,May 15 2020,1
1,Hertz Corporation,Automotive,May 22 2020,1
2,Frontier Communications Corp,Telecom,April 14 2020,1
3,Diamond Offshore Drilling,Oil and Gas,April 26 2020,1
4,McDermott Internation Inc,Oil and Gas,Jan 21 2020,1
5,CEC Entertainment( Chuck E Cheese's),Restaurant,June 24 2020,1
6,Neiman Marcus,Retail,May 7 2020,1
7,Latam Airlines,Aviation,May 25 2020,1
8,Whiting Petroleum Corporation,Oli and Gas,April 1 2020,1
9,Quorum Health Corporation,HealthCare,April 7 2020,1


In [ ]:
# now we want to pull in relevant data for each company
# for our bankrupt companies, we want to pull in news that is within 1 year of bankruptcy
# for our health companies, the timing a bit less important, but we will try to match to similar time periods

